In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing import Literal

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


In [3]:
model = ChatOpenAI(model="gpt-4o", api_key=os.getenv('openai-api-key'))

In [18]:
template = """You are a data scientist working for an e-commerce company. You have created a Tableau dashboard containing the following two graphs: 
1- a line plot of revenue for each day over the past 4 years 
2- a pie chart of cost broken down by different processes 

The following data is avaible: 

Sales (
datetime, 
revenue
)

Costs (
datetime,
category, 
amount
)

Tax (
datetime, 
amount, 
is_paid)

Note that one can use Tableau's functionalities to modify the graph. The only thing you cannot change is the data source.

A C-Level manager at this e-commerce company has the following questions: 
{question}

{options}

{format_instructions}"""

In [5]:
class FeasiblityCheck(BaseModel):
    is_feasible: Literal["yes", "partially", "no"] = Field(description="whether it is possible to answer the given question using the available graphs or data")

feasibility_parser = JsonOutputParser(pydantic_object=FeasiblityCheck)

In [6]:
feasibility_prompt_template = PromptTemplate(
    template=template, 
    input_variables=["question", "options"], 
    partial_variables={"format_instructions": feasibility_parser.get_format_instructions()}
)

feasibility_prompt_template

PromptTemplate(input_variables=['options', 'question'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"is_feasible": {"title": "Is Feasible", "description": "whether it is possible to answer the given question using the available graphs or data", "enum": ["yes", "partially", "no"], "type": "string"}}, "required": ["is_feasible"]}\n```'}, template="You are a data scientist working for an e-commerce company. You have created a Tableau dashboard containing the following two graphs: \n1- a line plot of revenue for each day over

In [7]:
chain = feasibility_prompt_template | model | feasibility_parser

In [8]:
question = "How much tax we pain the last month?"

options = """You have the following options to choose from:
* It is feasible, we can use one of the graphs mentioned above to answer the question.  
* It is partially feasible, given the data we can build a new graph that answers the question.
* It is not feasible, given the data available it is not possible to answer this question"""

chain.invoke({"question": question, "options": options})

{'is_feasible': 'partially'}

In [16]:
class SQLQuery(BaseModel):
    sql_query: str = Field(
        description="A runnable sql query that takes the available data and generates the data required to answer the question"
    )

parser = JsonOutputParser(pydantic_object=SQLQuery)

prompt_template = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions(),
        "options": "",
    },
)

chain = prompt_template | model | parser

In [19]:
question = "What is average amount of tax paid in the past 3 months?"

chain.invoke({"question": question})

{'sql_query': 'SELECT AVG(amount) AS average_tax_paid FROM Tax WHERE is_paid = TRUE AND datetime >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH)'}